In [ ]:
import pandas as pd
wine = pd.read_csv('https://bit.ly/wine_csv_data')

In [ ]:
data = wine[['alcohol','sugar','pH']]
target = wine['class']

class 열 target으로 사용, 나머지 열 특성으로 사용

In [ ]:
from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = train_test_split(
    data, target, test_size=0.2, random_state=42)

In [ ]:
sub_input, val_input, sub_target, val_target = train_test_split(
    train_input, train_target, test_size=0.2, random_state=42)

trainjnput과 train_target을 다시 train_test_split() 힘'수에 넣어 훈련 세트 sub_input, sub_target과 검증 세트 val_nput, val_target을 만듬.
test_size 매개변수를 0.2로 지정하여 train_inp니t의 약 20%를 val_input으로 만듬

In [ ]:
print(sub_input.shape, val_input.shape)

(4157, 3) (1040, 3)


훈련세트와 검증세트 크기 확인...
원래 5197개였던 훈련세트 4157개로 줄고 검증세트는 1040개 됨

In [ ]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(random_state=42)
dt.fit(sub_input, sub_target)
print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))

0.9971133028626413
0.864423076923077


subjnput, sub_target과 valjnp니t, val_target을 사용해 모델을 만들고 평가함

In [ ]:
from sklearn.model_selection import cross_validate
scores = cross_validate(dt, train_input, train_target)
print(scores)

{'fit_time': array([0.01886392, 0.01148248, 0.01064348, 0.01001263, 0.0097363 ]), 'score_time': array([0.00472212, 0.00288486, 0.00247145, 0.00229001, 0.00231814]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


In [ ]:
import numpy as np
print(np.mean(scores['test_score']))

0.855300214703487


검증폴드의 점수

In [ ]:
from sklearn.model_selection import StratifiedKFold
scores = cross_validate(dt, train_input, train_target, cv=StratifiedKFold())
print(np.mean(scores['test_score']))

0.855300214703487


cross_validate() 함수 는 기본적으로 회귀 모델일 경우 KFold 분할기를 사용하고 분류 모델일 경우 타깃 클래스를 골고루 나누기 위해 StratifiedKFold를 사용

In [ ]:
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_validate(dt, train_input, train_target, cv=splitter)
print(np.mean(scores['test_score']))

0.8574181117533719


In [ ]:
from sklearn.model_selection import GridSearchCV
params = {'min_impurity_decrease': [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}

In [ ]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42),params, n_jobs=-1)

In [ ]:
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

In [ ]:
dt = gs.best_estimator_
print(dt.score(train_input, train_target))

0.9615162593804117


GridSearchCV(그리드 서치) 를 사용해 모델을 튜닝한 뒤,
최적의 모델을 꺼내서 훈련 세트 성능을 확인

In [ ]:
print(gs.best_params_)

{'min_impurity_decrease': 0.0001}


GridSearchCV로 찾은 최적 모델(dt) 을 사용해서
훈련 세트의 정확도를 출력

In [ ]:
print(gs.cv_results_['mean_test_score'])

[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]


In [ ]:
print(gs.cv_results_['params'][gs.best_index_])

{'min_impurity_decrease': 0.0001}


GridSearchCV가 수행한 모든 실험 중에서 가장 좋은 결과를 낸 하이퍼파라미터 조합을 출력

In [ ]:
params = {'min_impurity_decrease': np.arange(0.0001, 0.001, 0.0001),
          'max_depth': range(5, 20, 1),
          'min_samples_split': range(2, 100, 10)
          }

결정 트리(DecisionTreeClassifier) 의 하이퍼파라미터 탐색 범위를 설정하는 부분

In [ ]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'min_impurity_decrease': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
       0.0009]),
                         'min_samples_split': range(2, 100, 10)})

In [ ]:
print(gs.best_params_)

{'max_depth': 14, 'min_impurity_decrease': np.float64(0.0004), 'min_samples_split': 12}


In [ ]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8683865773302731


In [ ]:
from scipy.stats import uniform, randint

싸이파이의 stats 서브 패키지에 있는 uniform과 randint 클래스는 모두 주어진 범위에서 고르게 값을 뽑음

In [ ]:
rgen = randint(0,10)
rgen.rvs(10)

array([8, 1, 2, 9, 3, 9, 2, 8, 8, 7])

0에서 10 사이의 범위를 갖는 randint 객체를 만 들고 10개의 숫자를 샘플링 함

In [ ]:
np.unique(rgen.rvs(1000), return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([ 95, 108, 101, 106, 108, 104,  97, 105,  82,  94]))

1,000개를 샘플링해서 각숫자의 개수를 세기

In [ ]:
ugen = uniform(0,1)
ugen.rvs(10)

array([0.78130969, 0.08309922, 0.10648514, 0.51687815, 0.01657604,
       0.90853857, 0.84889032, 0.47368383, 0.08990267, 0.91048102])

uniform 클래스의 사용해 0~1 사이에서 10개의 실수를 추출

In [ ]:
params = {'min_impurity_decrease': uniform(0.0001, 0.001),
          'max_depth': randint(20,50),
          'min_samples_split': randint(2, 25),
          'min_samples_leaf': randint(1, 25),
          }

min_imputiry_decrease는 0.00()1 에서 0.001 사이의 실숫값을 샘플링함 max_depth
는 20에서 50 사이의 정수, min_samples_split은 2에서 25 사이의 정수, min_samples_
leaf는 1에서 25 사이의 정수를 샘플링 함

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
rs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), params,
                        n_iter=100, n_jobs=-1, random_state=42)
rs.fit(train_input, train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7ea31ab6a4e0>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x7ea32209a8d0>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7ea31ab6a0f0>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7ea31ab685f0>},
                   random_state=42)

위 params에 정의된 매개변수 범위에서 총 100번 (njter 매개변수)을 샘플링하여 교차 검증을 수
행하고 최적의 매개변수 조합을 찾음

In [ ]:
print(rs.best_params_)

{'max_depth': 39, 'min_impurity_decrease': np.float64(0.00034102546602601173), 'min_samples_leaf': 7, 'min_samples_split': 13}


최적의 매개변수 조합 출력

In [ ]:
print(np.max(rs.cv_results_['mean_test_score']))

0.8695428296438884


최고의 교차 검증 점수 확인

In [ ]:
dt = rs.best_estimator_
print(dt.score(test_input, test_target))

0.86


best_estimator_속성에 저장된 최적의 모델 갖고 테스트 세트 성능 확인